<a href="https://colab.research.google.com/github/kamleshcode/intern-project/blob/main/ML_Pipeline_with_ZenML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Key Concepts:**ML Pipelines Steps

In this notebook. we will learn how to easily convert existing ML code into ML pipelines using ZenML.

In [1]:
%pip install "zenml[server]"
!zenml integration install sklearn —y
%pip install pyparsing==2.4.2 #required for Colab
import IPython
#automatically restart kernel
IPython.Application.instance().kernel.do_shutdown(restart=True)

NumExpr defaulting to 2 threads.
All required packages for integration 'sklearn' are already installed.
Unable to find integration '—y'.
  Using cached pyparsing-2.4.2-py2.py3-none-any.whl (65 kB)
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 2.4.7
    Uninstalling pyparsing-2.4.7:
      Successfully uninstalled pyparsing-2.4.7


{'status': 'ok', 'restart': True}

In [25]:
NGROK_TOKEN = "2fiJ3BA1OUCSgEwNd52NpmZYppS_56uYRY9SMc8489WjeFmk6"

In [26]:
from zenml.environment import Environment
if Environment.in_google_colab():
  #install and authenticate ngrok
  !pip install pyngrok
  !ngrok authtoken {NGROK_TOKEN}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


ZenML Setup

In [27]:
!rm -rf .zen #to remove any existing file
!zenml init  # to initialize zenml repositary

NumExpr defaulting to 2 threads.
⠋ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
Setting the repo active workspace to 'default'.
Setting the repo active stack to default.
ZenML repository initialized at /content.
⠇ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.

The local active stack was initialized to 'default'. This local configuration will only take effect 
when you're running ZenML from the initialized repository root, or from a subdirectory. For more 
information on repositories and configurations, please visit 
https://docs.zenml.io/user-guide/starter-guide/understand-stacks.


Example Experimentation ML code

In [28]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn. datasets import load_digits
from sklearn.model_selection import train_test_split

def train_test() -> None:
  """Train and Test a scikit-learn SVC classifier on digits"""
  digits=load_digits()
  data=digits.images.reshape((len(digits.images),-1))
  X_train,X_test,Y_train,Y_test = train_test_split(
      data, digits.target, test_size=0.2, shuffle=False
  )
  model=SVC(gamma=0.001)
  model.fit(X_train,Y_train)
  test_acc=model.score(X_test,Y_test)
  print(f"Test accuracy:{test_acc}")
train_test()


Test accuracy:0.9583333333333334


Turning experiments into ML Pipelines with ZenML

Importer --> SVC Trainer --> Evaluator

In [29]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

@step
def importer() ->Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[np.ndarray, "Y_train"],
    Annotated[np.ndarray, "Y_test"]
]:

    """Load digit Dataset as Numpy Arrays"""
    digits=load_digits()
    data=digits.images.reshape((len(digits.images),-1))
    X_train,X_test,Y_train,Y_test = train_test_split(
      data, digits.target, test_size=0.2, shuffle=False
    )
    return X_train, X_test,Y_train,Y_test


@step
def svc_trainer(
    X_train: np.ndarray,
    Y_train: np.ndarray,
 )->ClassifierMixin:
    """Train as sklearn SVC classifier."""
    model=SVC(gamma=0.001)
    model.fit(X_train, Y_train)
    return model

@step
def evaluator(
    X_test:np.ndarray,
    Y_test:np.ndarray,
    model:ClassifierMixin,
)->float:
   """Calculate the test set accuracy of an sklearn model."""
   test_acc = model.score(X_test,Y_test)
   print(f"Test accuracy:{test_acc}")
   return test_acc

Similarly we can use ZenML @pipeline decorator to connect all our steps into an ML pipeline.
Note that the pipeline definition does not depend on the concrete step functions we defined above it merely establishes a recipe for how data
moves through the steps. This means we can replace steps as we wish, e.g. to run the Same pipeline with different models to compare their
performances.

In [30]:
from zenml import pipeline

@pipeline
def digit_pipeline():
  """Links all the steps together in a pipeline"""
  X_train,X_test,Y_train,Y_test = importer()
  model = svc_trainer(X_train=X_train, Y_train=Y_train)
  evaluator(X_test=X_test,Y_test=Y_test,model=model)

Running ZenML Pipelines

Finally, we initialize our pipeline with concrete step functions and call the run() method to run it.

In [31]:
digits_svc_pipeline=digit_pipeline()
#digits_svc_pipeline.run(unlisted=True)


Initiating a new run for the pipeline: digit_pipeline.
Reusing registered pipeline version: (version: 1).
Executing a new run.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.
Using cached version of importer.
Step importer has started.
Using cached version of svc_trainer.
Step svc_trainer has started.
Using cached version of evaluator.
Step evaluator has started.
Pipeline run has finished in 0.742s.


And that's it. we just built and ran our first ML pipeline! Great job!

You can now visualize the pipeline run in ZenMl's dashboard. TO do so, run zenml up to spin up a ZenML dashboard locally, log in with
username default and empty password, and navigate to the "Runs" tab in the 'Pipelines' section.

In [ ]:
from zenml.environment import Environment
def start_zenml_dashboard(port=8237):
  if Environment.in_google_colab():
    from pyngrok import ngrok
    public_url = ngrok.connect(port)
    print(f"/xlb[31mIn Colab, use this URL insted: {public_url}!\x1b[Om")
    !zenml up --blocking --port {port}

  else:
    !zenml up --port {port}

start_zenml_dashboard()


INFO:pyngrok.ngrok:Opening tunnel named: http-8237-f5ad5396-8376-4476-87d7-2469293824af


Opening tunnel named: http-8237-f5ad5396-8376-4476-87d7-2469293824af


INFO:pyngrok.process.ngrok:t=2024-04-28T04:51:45+0000 lvl=info msg="no configuration paths supplied"


t=2024-04-28T04:51:45+0000 lvl=info msg="no configuration paths supplied"


INFO:pyngrok.process.ngrok:t=2024-04-28T04:51:45+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml


...

t=2024-04-28T04:51:45+0000 lvl=info msg=start pg=/api/tunnels id=37c54939204a990a


INFO:pyngrok.process.ngrok:t=2024-04-28T04:51:45+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8237-f5ad5396-8376-4476-87d7-2469293824af addr=http://localhost:8237 url=https://cc85-34-23-193-73.ngrok-free.app


t=2024-04-28T04:51:45+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8237-f5ad5396-8376-4476-87d7-2469293824af addr=http://localhost:8237 url=https://cc85-34-23-193-73.ngrok-free.app


INFO:pyngrok.process.ngrok:t=2024-04-28T04:51:45+0000 lvl=info msg=end pg=/api/tunnels id=37c54939204a990a status=201 dur=41.991574ms


/xlbIn Colab, use this URL insted: NgrokTunnel: "https://cc85-34-23-193-73.ngrok-free.app" -> "http://localhost:8237"!m
t=2024-04-28T04:51:45+0000 lvl=info msg=end pg=/api/tunnels id=37c54939204a990a status=201 dur=41.991574ms
NumExpr defaulting to 2 threads.
The local ZenML dashboard is about to deploy in a blocking process. You can connect to it using the 
'default' username and an empty password.
Deploying a local ZenML server with name 'local'.
Starting ZenML Server as blocking process... press CTRL+C once to stop it.
INFO:     Started server process [29931]
INFO:     Waiting for application startup.
Not writing the global configuration to disk in a ZenML server environment.
Not writing the global configuration to disk in a ZenML server environment.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8237 (Press CTRL+C to quit)
